In [1]:
import logging
reload(logging)
log_fmt = '%(asctime)-9s %(levelname)-8s: %(message)s'
logging.basicConfig(format=log_fmt)

# Change to info once the notebook runs ok
#logging.getLogger().setLevel(logging.DEBUG)

In [2]:
%pylab inline

import datetime
import devlib
import os
import json
import pandas as pd
import re
import subprocess
import trappy
from trappy.plotter.Utils import get_trace_event_data

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

# Support to access the remote target
#import devlib
#from env import TestEnv

from env import TestEnv
from executor import Executor

Populating the interactive namespace from numpy and matplotlib


# Target Configuration

In [3]:
# Setup a target configuration
env = TestEnv({
    
    # Target platform and board
    "platform"    : 'linux',
    "board"       : 'aboard',
    
    # Target board IP/MAC address
    "host"        : '192.168.0.1',
    
    # Login credentials
    "username"    : 'brendan',
    "password"    : '',
        
    # Folder where all the results will be collected
    "results_dir" : "ExecutorExample",
        
    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "cpu_frequency",
         ],
         "buffsize" : 80 * 1024,
    },
            
    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'perf' ],
    
    # Modules required by these experiments
    "modules"     : [ 'bl', 'cpufreq' ],

})

06:16:08  INFO    :         Target - Using base path: /home/bjackman/sources/lisa
06:16:08  INFO    :         Target - Loading custom (inline) target configuration
06:16:08  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq']
06:16:08  INFO    :         Target - Connecting linux target:
06:16:08  INFO    :         Target -   username : brendan
06:16:08  INFO    :         Target -       host : 192.168.0.1
06:16:08  INFO    :         Target -   password : 
06:16:08  INFO    :         Target - Connection settings:
06:16:08  INFO    :         Target -    {'username': 'brendan', 'host': '192.168.0.1', 'password': ''}
06:16:14  INFO    :         Target - Initializing target workdir:
06:16:14  INFO    :         Target -    /home/brendan/devlib-target
06:16:18  INFO    :         Target - Topology:
06:16:18  INFO    :         Target -    [[0, 3, 4, 5], [1, 2]]
06:16:23  INFO    :         FTrace - Enabled tracepoints:
06:16:23  INFO    :         FTrace -   sched_switch
06:16:23 

# Tests Configuration

In [ ]:
executor = Executor(env, {
    # Platform configurations to test
    "confs" : [
        {
            "tag"            : "base",
            "flags"          : "ftrace",           # Enable FTrace events
            "sched_features" : "NO_ENERGY_AWARE",  # Disable EAS
            "cpufreq"        : {                   # Use PERFORMANCE CpuFreq
                "governor" : "performance",
            },
        },
        {
            "tag"            : "eas",
            "flags"          : "ftrace",           # Enable FTrace events
            "sched_features" : "ENERGY_AWARE",     # Enable EAS
            "cpufreq"        : {                   # Use PERFORMANCE CpuFreq
                "governor" : "performance",
            },
        },
    ],
    
    # Workloads to run (on each platform configuration)
    "wloads" : {
        # Run hackbench with 1 group using pipes
        "perf" : {
            "type" : "perf_bench",
            "conf" : {
                "class" : "messaging",
                "params" : {
                    "group" :    1,
                    "loop"  :   10,
                    "pipe"  : True,
                    "thread": True,
                }
            }
        },
        # Run a 20% duty-cycle periodic task
        "rta" : {
            "type" : "rt-app",
            "loadref" : "big",
            "conf" : {
                "class"  : "profile",
                "params"  : {
                    "p20" : {
                        "kind"   : "Periodic",
                        "params" : {
                            "duty_cycle_pct" : 20,
                         },
                    },
                },
            },
        },
    },
    
    # Number of iterations for each workload
    "iterations" : 1,
})

06:16:23  INFO    :         Target - Loading custom (inline) test configuration
06:16:23  INFO    : 
06:16:23  INFO    : ################################################################################
06:16:23  INFO    :       Executor - Experiments configuration
06:16:23  INFO    : ################################################################################
06:16:23  INFO    :       Executor - Configured to run:
06:16:23  INFO    :       Executor -     2 target configurations:
06:16:23  INFO    :       Executor -       base, eas
06:16:23  INFO    :       Executor -     2 workloads (1 iterations each)
06:16:23  INFO    :       Executor -       rta, perf
06:16:23  INFO    :       Executor - Total: 4 experiments
06:16:23  INFO    :       Executor - Results will be collected under:
06:16:23  INFO    :       Executor -       /home/bjackman/sources/lisa/results/ExecutorExample


# Tests execution

In [ ]:
executor.run()

06:16:23  INFO    : 
06:16:23  INFO    : ################################################################################
06:16:23  INFO    :       Executor - Experiments execution
06:16:23  INFO    : ################################################################################
06:16:23  INFO    : 
06:16:23  INFO    : ================================================================================
06:16:23  INFO    :   TargetConfig - configuring target for [base] experiments
06:16:25  INFO    :  SchedFeatures - Set scheduler feature: NO_ENERGY_AWARE
06:16:26  INFO    :        CPUFreq - Configuring all CPUs to use [performance] governor
06:16:27  INFO    :          WlGen - Setup new workload rta
06:16:27  INFO    :          RTApp - Workload duration defined by longest task
06:16:27  INFO    :          RTApp - Default policy: SCHED_OTHER
06:16:27  INFO    :          RTApp - ------------------------
06:16:27  INFO    :          RTApp - task [task_p200], sched: using default policy
06:1

In [7]:
!tree {executor.te.res_dir}

/home/brejac01/sources/lisa/results/ExecutorExample
├── perf_bench_messaging:base:perf
│   ├── 1
│   │   ├── output.log
│   │   ├── performance.json
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── perf_bench_messaging:eas:perf
│   ├── 1
│   │   ├── output.log
│   │   ├── performance.json
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── rtapp:base:rta
│   ├── 1
│   │   ├── output.log
│   │   ├── rta_00.json
│   │   ├── rt-app-task_p200-0.log
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
└── rtapp:eas:rta
    ├── 1
    │   ├── output.log
    │   ├── rta_00.json
    │   ├── rt-app-task_p200-0.log
    │   └── trace.dat
    ├── kernel.config
    ├── kernel.version
    └── platform.json

8 directories, 26 files
